In [ ]:
import pandas as pd
import numpy as np
import importlib
from tqdm import tqdm
import gc
import os
from skimage.metrics import structural_similarity as ssim

# plot and image libraries
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import plotly.graph_objects as go
from skimage import io, color
from io import BytesIO

In [ ]:
from claspy.data_loader import load_tssb_dataset
from claspy.data_loader import load_has_dataset

In [ ]:
# custom utils
import mt_utils as ut

In [ ]:
importlib.reload(ut)

In [ ]:
tssb = load_tssb_dataset()
hasc = load_has_dataset()

In [ ]:
tssb_ds = pd.read_pickle('results/zwischenergebnisse/main_experiment_tssb.pkl')
hasc_ds = pd.read_pickle('results/zwischenergebnisse/main_experiment_hasc.pkl')

# Functions

In [ ]:
columns = ['Algorithm','Orig_TS_ID','compression','Orig_TS','DS_TS','ClaSP_score','DSSIM','FTbSim']
algos_sorted_dsOnly = ["EveryNth","MinMax","M4","LTTB","MinMaxLTTB","LTD"]

In [ ]:
def preprocess_image(image):
    # ensure image is rendered in Greyscales
    if image.ndim == 3 and image.shape[2] == 4:
        image = image[..., :3] # only use first three channels (RGB)
    if image.ndim == 3:
        image = color.rgb2gray(image)
    return image

In [ ]:
def plot_to_image(ts_arr):
    # create plot image of time series and return the image as numpy array
    image_list=[]
    for ts in ts_arr:
        fig, ax = plt.subplots()
        ax.plot(ts, color='black')
        ax.axis('off')
        buf = BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        plt.close('all')
        buf.seek(0)
        image = io.imread(buf)
        buf.close()
        image_list.append(preprocess_image(image))
        images = np.asarray(image_list)
    del image, image_list
    gc.collect()
    return images

In [ ]:
def calc_dssim(image1, image2): 
    # SSIM
    ssim_value = ssim(image1, image2, data_range=image1.max() - image1.min())
    
    # DSSIM
    dssim_value = (1 - ssim_value) / 2
    return dssim_value

In [ ]:
def eval_downsampling_performances(df,output_file="temp_results.csv"):
    pd.DataFrame(columns=columns).to_csv(output_file, index=False)
    orig_data = df[df.Algorithm=='None']
    for index, row in tqdm(orig_data.iterrows(), total=len(orig_data), desc="Processing rows"):
        ts1 = ut.prepare_ts(row.Time_Series)           
        images1 = plot_to_image(ts1)            
        for algo in algos_sorted_dsOnly:
            algo_row = df[(df.Algorithm==algo)&(df.orig_TS_ID==row.orig_TS_ID)].T.squeeze()
            ts2 = ut.prepare_ts(algo_row.Time_Series)
            images2 = plot_to_image(ts2)
            
            dssims = [calc_dssim(images1[dim], images2[dim]) for dim in range(ts1.shape[0])]
            ftbsims = [ut.z_ED_fft(ts1[dim], ts2[dim]) for dim in range(ts1.shape[0])]
 
            result_row = {
                "Algorithm": algo,
                "Orig_TS_ID": row.orig_TS_ID,
                "compression": algo_row.compression,
                "Orig_TS": row.Time_Series,
                "DS_TS": ts2.T if ts2.shape[0] > 1 else ts2.squeeze(),
                "ClaSP_score": algo_row.score,
                "DSSIM": float(np.mean(dssims)),
                "FTbSim": float(np.mean(ftbsims)),
            }

            pd.DataFrame([result_row]).to_csv(output_file, mode='a', index=False, header=False)

            del ts2, images2, dssims, ftbsims
            gc.collect()
        
        del ts1, images1
        gc.collect()

    final_df = pd.read_csv(output_file)

    os.remove(output_file)
    return final_df

In [ ]:
def plot_simMeasure_score(df,measure,xaxis_title,title, filepath=None):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[measure], y=df.ClaSP_score,
                                mode='markers',
                                marker= dict(
                                    color='grey',
                                    size=5,
                                    line=dict(
                                        color='black',
                                        width=1
                                    )
                                )
                            )
                 )
            
    fig.update_yaxes(title_text="Covering Score")
    fig.update_xaxes(title_text=xaxis_title)
        
    fig.update_layout(height=400,
                          font=dict(
                              family="Arial",
                              size=12,
                              color="black"
                          ),
                          title=dict(
                              text=title
                          ),
                          margin=dict(l=0, r=0, t=30, b=0)
                         )
    
    if filepath:
        fig.write_image(filepath, scale=1, width=1000, height=400)
    
    fig.show()

In [ ]:
def score_comparison_matrix(df, savepath=None):
    dssim_df = df[df.groupby(by='Orig_TS_ID')['DSSIM'].transform(min) == df['DSSIM']]
    mean1 = dssim_df.ClaSP_score.mean()
    median1 = dssim_df.ClaSP_score.median()
    std1 = dssim_df.ClaSP_score.std()

    ftbsim_df = df[df.groupby(by='Orig_TS_ID')['FTbSim'].transform(min) == df['FTbSim']]
    mean2 = ftbsim_df.ClaSP_score.mean()
    median2 = ftbsim_df.ClaSP_score.median()
    std2 = ftbsim_df.ClaSP_score.std()

    out = pd.DataFrame(columns=['DSSIM','FTbSim'], index=['mean','median','std'])
    out.loc['mean']=[mean1,mean2]
    out.loc['median']=[median1,median2]
    out.loc['std']=[std1,std2]
                       
    if(savepath):
        out.to_excel(savepath)

    return out

# Results

## TSSB

In [ ]:
ds_performances_tssb_df = eval_downsampling_performances(tssb_ds)

In [ ]:
ds_performances_tssb_df.to_pickle('results/zwischenergebnisse/downsampling_performance_tssb.pkl')
ds_performances_tssb_df

In [ ]:
ds_performances_tssb_df[['Algorithm','DSSIM','FTbSim','ClaSP_score']].groupby(by='Algorithm').mean()

In [ ]:
plot_simMeasure_score(ds_performances_tssb_df,'DSSIM','DSSIM','(a) TSSB','results/figures/downsampling_performance_DSSIM_tssb.svg')

In [ ]:
plot_simMeasure_score(ds_performances_tssb_df,'FTbSim','Feature-based Similarity',
                      '', 'results/figures/downsampling_performance_FTbSim_tssb.svg')

In [ ]:
score_comparison_matrix_columns(ds_performances_tssb_df, savepath=None)

In [ ]:
score_comparison_matrix(ds_performances_tssb_df, savepath='results/tables/summary_statistics_dsperformance_tssb.xlsx')

In [ ]:
# this is what is possible when using the best settings, e.g. Algorithm combined with cr, upscaling and window size
# issue: how to find a prediction that approaches this setting
tmp_tssb = ds_performances_tssb_df[ds_performances_tssb_df.groupby(by='Orig_TS_ID')['ClaSP_score'].transform(max) == ds_performances_tssb_df['ClaSP_score']]
tmp_tssb[['Algorithm','ClaSP_score']].groupby(by='Algorithm').mean().T

print('Mean: {0}, Median: {1}, Std: {2}'.format(tmp_tssb.ClaSP_score.mean(),tmp_tssb.ClaSP_score.median(),tmp_tssb.ClaSP_score.std()))

## HASC

In [ ]:
ds_performances_hasc_df = eval_downsampling_performances(hasc_ds)

In [ ]:
ds_performances_hasc_df.to_pickle('results/zwischenergebnisse/downsampling_performance_hasc.pkl')
ds_performances_hasc_df

In [ ]:
plot_simMeasure_score(ds_performances_hasc_df,'DSSIM','DSSIM','(b) HASC', 'results/figures/downsampling_performance_DSSIM_hasc.svg')

In [ ]:
plot_simMeasure_score(ds_performances_hasc_df,'FTbSim','Feature-based Similarity','', 'results/figures/downsampling_performance_FTbSim_hasc.svg')

In [ ]:
score_comparison_matrix(ds_performances_hasc_df, savepath='results/tables/summary_statistics_dsperformance_hasc.xlsx')

In [ ]:
# what is possible with best settings
tmp_hasc = ds_performances_hasc_df[ds_performances_hasc_df.groupby(by='Orig_TS_ID')['ClaSP_score'].transform(max) == ds_performances_hasc_df['ClaSP_score']]
tmp_hasc[['Algorithm','ClaSP_score']].groupby(by='Algorithm').mean().T

print('Mean: {0}, Median: {1}, Std: {2}'.format(tmp_hasc.ClaSP_score.mean(),tmp_hasc.ClaSP_score.median(),tmp_hasc.ClaSP_score.std()))